In [1]:
# Small Case
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import *

result_df = pd.DataFrame()

parameters = [
    "BQPCuts",
    "CliqueCuts",
    "CoverCuts",
    "FlowCoverCuts",
    "FlowPathCuts",
    "GUBCoverCuts",
    "ImpliedCuts",
    "InfProofCuts",
    "LiftProjectCuts",
    "MIPSepCuts",
    "MIRCuts",
    "ModKCuts",
    "NetworkCuts",
    "ProjImpliedCuts",
    "PSDCuts",
    "RelaxLiftCuts",
    "RLTCuts",
    "StrongCGCuts",
    "SubMIPCuts",
    "ZeroHalfCuts"
]
for parameter in parameters:
    for parakey in [-1,0,1,2]:
            df = pd.read_csv(r"D:\Vibhu_Git_Projects\Optimization\Concert_TSPTW_Optimization\data\TSPTW_150.txt")
            all_vals = []
            for i in range(df.shape[0]):

                vals = [val for val in df.iloc[i,0].strip().split(" ") if val != ""]
                all_vals.append(vals)

            df_data = pd.DataFrame(all_vals, columns = ["CUST NO.", "X", "Y", "DEMAND", "READY TIME", "DUE DATE", "SERVICE TIME"])
            df_data = df_data.iloc[:-1,:]
            df_data["COORDINATES"] = df_data.apply(lambda row: (row["X"], row["Y"]), axis=1)
            df_data["DEMAND"] = np.random.randint(1,5,df_data.shape[0])
            df_data["SERVICE TIME"] = np.random.randint(1,9,df_data.shape[0])
            df_data = df_data.drop(columns=["X", "Y"])
            df_small = df_data.iloc[1:].sample(n = 80, replace = False, random_state=42).copy()
            initial = pd.DataFrame([df_data.iloc[0].to_list()], columns = df_data.columns)
            df_small = pd.concat([initial, df_small]).reset_index(drop=True)
            ### Create Distance and Time Matrices
            import math
            # Compute pairwise times matrix

            def distance(df, col, row1, row2):
                c1 = df.loc[row1, col]
                c2 = df.loc[row2, col]
                diff = (float(c1[0])-float(c2[0]), float(c1[1])-float(c2[1]))
                return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])

            small_dist = []
            
            for row1 in range(df_small.shape[0]):
                
                row_vals = []
                for row2 in range(df_small.shape[0]):
    
                    row_vals.append(distance(df_small, "COORDINATES", row1, row2))
                small_dist.append(row_vals)

            dist_small = np.asarray(small_dist)
            ### Model Implementation - Small
            I = len(dist_small)
            J = len(dist_small[0])

            # Start Times
            A = df_small["READY TIME"].astype(float).to_numpy()

            # End Times
            B = df_small["DUE DATE"].astype(float).to_numpy()

            # distance matrix
            D = dist_small
            probS = gp.Model("TSPTW - Concerts - Small")

            X = probS.addVars(I, J, vtype=GRB.BINARY, name=[f"({i}), ({j})" for i in range(I) for j in range(J)])
            TI = probS.addVars(I, vtype=GRB.INTEGER, name=[f"TI_{i}" for i in range(I)])
            TJ = probS.addVars(J, vtype=GRB.INTEGER, name=[f"TJ_{j}" for j in range(J)])

            probS.setObjective(gp.quicksum(X[i,j]*D[i][j] for i in range(I) for j in range(J)), GRB.MINIMIZE)
            for i in range(I):
                # Initiation of Arrival to first node
                probS.addConstr(TI[i] - D[0][i]*X[0,i] >= 0)

                # start of time window
                probS.addConstr(TI[i] >= A[i])

                # end of time window
                probS.addConstr(TJ[i] <= B[i])

                # only one per row
                probS.addConstr(gp.quicksum(X[i,j] for j in range(J)) == 1, name = "row" + str(i))

                # cannot go along diagonals
                probS.addConstr(X[i,i] == 0)

            for j in range(J):
                # only one per column
                probS.addConstr(gp.quicksum(X[i,j] for i in range(I)) == 1, name = "col" + str(j))
            # Sub tour Elimination
            for i in range(I):
                for j in range(J):
                    if i == j:
                        continue
                    else:
                        probS.addConstr(TI[i] - TJ[j] + (B[i] - A[j] + D[i][j])*X[i,j] <= B[i] - A[j], name = "row" + str(i) + " - col" + str(j))

            probS.setParam(parameter, parakey)
            probS.optimize()

            params = {
            'Usecase' : 'Small',
            'Parameter': parameter,
            'Parameter Value': parakey,
            'Obj Value': probS.ObjVal,
            'Runtime' : probS.Runtime
            }
            # Convert the dictionary to a dataframe
            params_df = pd.DataFrame(params, index=[0])
            print(params_df)
            # Append the dataframe to the result_df
            result_df = result_df.append(params_df, ignore_index=True)
            
parameters = [
    "CutPasses",
    "CutAggPasses",
    "GomoryPasses"
]
for parameter in parameters:
    for parakey in range(100):
            df = pd.read_csv(r"D:\Vibhu_Git_Projects\Optimization\Concert_TSPTW_Optimization\data\TSPTW_150.txt")
            all_vals = []
            for i in range(df.shape[0]):

                vals = [val for val in df.iloc[i,0].strip().split(" ") if val != ""]
                all_vals.append(vals)

            df_data = pd.DataFrame(all_vals, columns = ["CUST NO.", "X", "Y", "DEMAND", "READY TIME", "DUE DATE", "SERVICE TIME"])
            df_data = df_data.iloc[:-1,:]
            df_data["COORDINATES"] = df_data.apply(lambda row: (row["X"], row["Y"]), axis=1)
            df_data["DEMAND"] = np.random.randint(1,5,df_data.shape[0])
            df_data["SERVICE TIME"] = np.random.randint(1,9,df_data.shape[0])
            df_data = df_data.drop(columns=["X", "Y"])
            df_small = df_data.iloc[1:].sample(n = 80, replace = False, random_state=42).copy()
            initial = pd.DataFrame([df_data.iloc[0].to_list()], columns = df_data.columns)
            df_small = pd.concat([initial, df_small]).reset_index(drop=True)
            ### Create Distance and Time Matrices
            import math
            # Compute pairwise times matrix

            def distance(df, col, row1, row2):
                c1 = df.loc[row1, col]
                c2 = df.loc[row2, col]
                diff = (float(c1[0])-float(c2[0]), float(c1[1])-float(c2[1]))
                return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])

            small_dist = []
            
            for row1 in range(df_small.shape[0]):
                
                row_vals = []
                for row2 in range(df_small.shape[0]):
    
                    row_vals.append(distance(df_small, "COORDINATES", row1, row2))
                small_dist.append(row_vals)

            dist_small = np.asarray(small_dist)
            ### Model Implementation - Small
            I = len(dist_small)
            J = len(dist_small[0])

            # Start Times
            A = df_small["READY TIME"].astype(float).to_numpy()

            # End Times
            B = df_small["DUE DATE"].astype(float).to_numpy()

            # distance matrix
            D = dist_small
            probS = gp.Model("TSPTW - Concerts - Small")

            X = probS.addVars(I, J, vtype=GRB.BINARY, name=[f"({i}), ({j})" for i in range(I) for j in range(J)])
            TI = probS.addVars(I, vtype=GRB.INTEGER, name=[f"TI_{i}" for i in range(I)])
            TJ = probS.addVars(J, vtype=GRB.INTEGER, name=[f"TJ_{j}" for j in range(J)])

            probS.setObjective(gp.quicksum(X[i,j]*D[i][j] for i in range(I) for j in range(J)), GRB.MINIMIZE)
            for i in range(I):
                # Initiation of Arrival to first node
                probS.addConstr(TI[i] - D[0][i]*X[0,i] >= 0)

                # start of time window
                probS.addConstr(TI[i] >= A[i])

                # end of time window
                probS.addConstr(TJ[i] <= B[i])

                # only one per row
                probS.addConstr(gp.quicksum(X[i,j] for j in range(J)) == 1, name = "row" + str(i))

                # cannot go along diagonals
                probS.addConstr(X[i,i] == 0)

            for j in range(J):
                # only one per column
                probS.addConstr(gp.quicksum(X[i,j] for i in range(I)) == 1, name = "col" + str(j))
            # Sub tour Elimination
            for i in range(I):
                for j in range(J):
                    if i == j:
                        continue
                    else:
                        probS.addConstr(TI[i] - TJ[j] + (B[i] - A[j] + D[i][j])*X[i,j] <= B[i] - A[j], name = "row" + str(i) + " - col" + str(j))

            probS.setParam(parameter, parakey)
            probS.optimize()

            params = {
            'Usecase' : 'Small',
            'Parameter': parameter,
            'Parameter Value': parakey,
            'Obj Value': probS.ObjVal,
            'Runtime' : probS.Runtime
            }
            # Convert the dictionary to a dataframe
            params_df = pd.DataFrame(params, index=[0])
            print(params_df)
            # Append the dataframe to the result_df
            result_df = result_df.append(params_df, ignore_index=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-20
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6966 rows, 6723 columns and 32966 nonzeros
Model fingerprint: 0x1b1213af
Variable types: 0 continuous, 6723 integer (6561 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 6815 rows and 3764 columns
Presolve time: 0.03s
Presolved: 151 rows, 2959 columns, 5925 nonzeros
Variable types: 0 continuous, 2959 integer (2959 binary)

Root relaxation: objective 5.453112e+02, 161 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     545.3111592  545.3111

C:\Users\workV\AppData\Local\Temp\ipykernel_24204\4256306941.py:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(params_df, ignore_index=True)


Set parameter BQPCuts to value 0
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6966 rows, 6723 columns and 32966 nonzeros
Model fingerprint: 0x1b1213af
Variable types: 0 continuous, 6723 integer (6561 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 6815 rows and 3764 columns
Presolve time: 0.02s
Presolved: 151 rows, 2959 columns, 5925 nonzeros
Variable types: 0 continuous, 2959 integer (2959 binary)

Root relaxation: objective 5.453112e+02, 161 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     545.3111592  545.31116  0.00%     -    0s

Explored 1 nodes (161 simplex iterat

C:\Users\workV\AppData\Local\Temp\ipykernel_24204\4256306941.py:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(params_df, ignore_index=True)


Set parameter BQPCuts to value 1
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6966 rows, 6723 columns and 32966 nonzeros
Model fingerprint: 0x1b1213af
Variable types: 0 continuous, 6723 integer (6561 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 6815 rows and 3764 columns
Presolve time: 0.02s
Presolved: 151 rows, 2959 columns, 5925 nonzeros
Variable types: 0 continuous, 2959 integer (2959 binary)

Root relaxation: objective 5.453112e+02, 161 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     545.3111592  545.31116  0.00%     -    0s

Explored 1 nodes (161 simplex iterat

...

C:\Users\workV\AppData\Local\Temp\ipykernel_24204\4256306941.py:236: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(params_df, ignore_index=True)


Set parameter GomoryPasses to value 98
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6966 rows, 6723 columns and 32966 nonzeros
Model fingerprint: 0x1b1213af
Variable types: 0 continuous, 6723 integer (6561 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 6815 rows and 3764 columns
Presolve time: 0.02s
Presolved: 151 rows, 2959 columns, 5925 nonzeros
Variable types: 0 continuous, 2959 integer (2959 binary)

Root relaxation: objective 5.453112e+02, 161 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     545.3111592  545.31116  0.00%     -    0s

Explored 1 nodes (161 simplex 

C:\Users\workV\AppData\Local\Temp\ipykernel_24204\4256306941.py:236: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(params_df, ignore_index=True)


Set parameter GomoryPasses to value 99
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6966 rows, 6723 columns and 32966 nonzeros
Model fingerprint: 0x1b1213af
Variable types: 0 continuous, 6723 integer (6561 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 6815 rows and 3764 columns
Presolve time: 0.01s
Presolved: 151 rows, 2959 columns, 5925 nonzeros
Variable types: 0 continuous, 2959 integer (2959 binary)

Root relaxation: objective 5.453112e+02, 161 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     545.3111592  545.31116  0.00%     -    0s

Explored 1 nodes (161 simplex 

C:\Users\workV\AppData\Local\Temp\ipykernel_24204\4256306941.py:236: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(params_df, ignore_index=True)


In [2]:
result_df

,Usecase,Parameter,Parameter Value,Obj Value,Runtime
0,Small,BQPCuts,-1,545.311159,0.073111
1,Small,BQPCuts,0,545.311159,0.055534
2,Small,BQPCuts,1,545.311159,0.059889
3,Small,BQPCuts,2,545.311159,0.050573
4,Small,CliqueCuts,-1,545.311159,0.047857
...,...,...,...,...,...
375,Small,GomoryPasses,95,545.311159,0.055563
376,Small,GomoryPasses,96,545.311159,0.044853
377,Small,GomoryPasses,97,545.311159,0.039980
378,Small,GomoryPasses,98,545.311159,0.059553


In [3]:
#Medium Case
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import *
parameters = [
    "BQPCuts",
    "CliqueCuts",
    "CoverCuts",
    "FlowCoverCuts",
    "FlowPathCuts",
    "GUBCoverCuts",
    "ImpliedCuts",
    "InfProofCuts",
    "LiftProjectCuts",
    "MIPSepCuts",
    "MIRCuts",
    "ModKCuts",
    "NetworkCuts",
    "ProjImpliedCuts",
    "PSDCuts",
    "RelaxLiftCuts",
    "RLTCuts",
    "StrongCGCuts",
    "SubMIPCuts",
    "ZeroHalfCuts"
]
for parameter in parameters:
    for parakey in [-1,0,1,2]:


        ### Read in Source Data and Create 3 Scenarios
        df = pd.read_csv(r"D:\Vibhu_Git_Projects\Optimization\Concert_TSPTW_Optimization\data\TSPTW_150.txt")


        all_vals = []
        for i in range(df.shape[0]):
            vals = [val for val in df.iloc[i,0].strip().split(" ") if val != ""]
            all_vals.append(vals)

        df_data = pd.DataFrame(all_vals, columns = ["CUST NO.", "X", "Y", "DEMAND", "READY TIME", "DUE DATE", "SERVICE TIME"])
        df_data = df_data.iloc[:-1,:]
        df_data["COORDINATES"] = df_data.apply(lambda row: (row["X"], row["Y"]), axis=1)
        df_data["DEMAND"] = np.random.randint(1,5,df_data.shape[0])
        df_data["SERVICE TIME"] = np.random.randint(1,9,df_data.shape[0])
        df_data = df_data.drop(columns=["X", "Y"])
        df_medium = df_data.iloc[1:].sample(n = 100, replace = False, random_state=42).copy()
        initial = pd.DataFrame([df_data.iloc[0].to_list()], columns = df_data.columns)
        df_medium = pd.concat([initial, df_medium]).reset_index(drop=True)
        ### Create Distance and Time Matrices
        import math
        # Compute pairwise times matrix

        def distance(df, col, row1, row2):
            c1 = df.loc[row1, col]
            c2 = df.loc[row2, col]
            diff = (float(c1[0])-float(c2[0]), float(c1[1])-float(c2[1]))
            return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])


        medium_dist = []
        for row1 in range(df_medium.shape[0]):
            row_vals = []
            for row2 in range(df_medium.shape[0]):
                row_vals.append(distance(df_medium, "COORDINATES", row1, row2))
            medium_dist.append(row_vals)
        dist_medium = np.asarray(medium_dist)


        I = len(dist_medium)
        J = len(dist_medium[0])

        # Start Times
        A = df_medium["READY TIME"].astype(float).to_numpy()

        # End Times
        B = df_medium["DUE DATE"].astype(float).to_numpy()

        # distance matrix
        D = dist_medium
        probM = gp.Model("TSPTW - Concerts - Medium")

        X = probM.addVars(I, J, vtype=GRB.BINARY, name=[f"({i}), ({j})" for i in range(I) for j in range(J)])
        TI = probM.addVars(I, vtype=GRB.INTEGER, name=[f"TI_{i}" for i in range(I)])
        TJ = probM.addVars(J, vtype=GRB.INTEGER, name=[f"TJ_{j}" for j in range(J)])

        probM.setObjective(gp.quicksum(X[i,j]*D[i][j] for i in range(I) for j in range(J)), GRB.MINIMIZE)
        for i in range(I):
            # Initiation of Arrival to first node
            probM.addConstr(TI[i] - D[0][i]*X[0,i] >= 0)

            # start of time window
            probM.addConstr(TI[i] >= A[i])

            # end of time window
            probM.addConstr(TJ[i] <= B[i])

            # only one per row
            probM.addConstr(gp.quicksum(X[i,j] for j in range(J)) == 1, name = "row" + str(i))

            # cannot go along diagonals
            probM.addConstr(X[i,i] == 0)

        for j in range(J):
            # only one per column
            probM.addConstr(gp.quicksum(X[i,j] for i in range(I)) == 1, name = "col" + str(j))
        # Sub tour Elimination
        for i in range(I):
            for j in range(J):
                if i == j:
                    continue
                else:
                    probM.addConstr(TI[i] - TJ[j] + (B[i] - A[j] + D[i][j])*X[i,j] <= B[i] - A[j], name = "row" + str(i) + " - col" + str(j))


        probM.setParam(parameter, parakey)
        probM.optimize()

        params = {
        'Usecase' : 'Medium',
        'Parameter': parameter,
        'Parameter Value': parakey,
        'Obj Value': probM.ObjVal,
        'Runtime' : probM.Runtime
        }
        # Convert the dictionary to a dataframe
        params_df = pd.DataFrame(params, index=[0])
 
        # Append the dataframe to the result_df
        result_df = result_df.append(params_df, ignore_index=True)
        
        
parameters = [
    "CutPasses",
    "CutAggPasses",
    "GomoryPasses"
]
for parameter in parameters:
    for parakey in range(100):
        ### Read in Source Data and Create 3 Scenarios
        df = pd.read_csv(r"D:\Vibhu_Git_Projects\Optimization\Concert_TSPTW_Optimization\data\TSPTW_150.txt")


        all_vals = []
        for i in range(df.shape[0]):
            vals = [val for val in df.iloc[i,0].strip().split(" ") if val != ""]
            all_vals.append(vals)

        df_data = pd.DataFrame(all_vals, columns = ["CUST NO.", "X", "Y", "DEMAND", "READY TIME", "DUE DATE", "SERVICE TIME"])
        df_data = df_data.iloc[:-1,:]
        df_data["COORDINATES"] = df_data.apply(lambda row: (row["X"], row["Y"]), axis=1)
        df_data["DEMAND"] = np.random.randint(1,5,df_data.shape[0])
        df_data["SERVICE TIME"] = np.random.randint(1,9,df_data.shape[0])
        df_data = df_data.drop(columns=["X", "Y"])
        df_medium = df_data.iloc[1:].sample(n = 100, replace = False, random_state=42).copy()
        initial = pd.DataFrame([df_data.iloc[0].to_list()], columns = df_data.columns)
        df_medium = pd.concat([initial, df_medium]).reset_index(drop=True)
        ### Create Distance and Time Matrices
        import math
        # Compute pairwise times matrix

        def distance(df, col, row1, row2):
            c1 = df.loc[row1, col]
            c2 = df.loc[row2, col]
            diff = (float(c1[0])-float(c2[0]), float(c1[1])-float(c2[1]))
            return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])


        medium_dist = []
        for row1 in range(df_medium.shape[0]):
            row_vals = []
            for row2 in range(df_medium.shape[0]):
                row_vals.append(distance(df_medium, "COORDINATES", row1, row2))
            medium_dist.append(row_vals)
        dist_medium = np.asarray(medium_dist)


        I = len(dist_medium)
        J = len(dist_medium[0])

        # Start Times
        A = df_medium["READY TIME"].astype(float).to_numpy()

        # End Times
        B = df_medium["DUE DATE"].astype(float).to_numpy()

        # distance matrix
        D = dist_medium
        probM = gp.Model("TSPTW - Concerts - Medium")

        X = probM.addVars(I, J, vtype=GRB.BINARY, name=[f"({i}), ({j})" for i in range(I) for j in range(J)])
        TI = probM.addVars(I, vtype=GRB.INTEGER, name=[f"TI_{i}" for i in range(I)])
        TJ = probM.addVars(J, vtype=GRB.INTEGER, name=[f"TJ_{j}" for j in range(J)])

        probM.setObjective(gp.quicksum(X[i,j]*D[i][j] for i in range(I) for j in range(J)), GRB.MINIMIZE)
        for i in range(I):
            # Initiation of Arrival to first node
            probM.addConstr(TI[i] - D[0][i]*X[0,i] >= 0)

            # start of time window
            probM.addConstr(TI[i] >= A[i])

            # end of time window
            probM.addConstr(TJ[i] <= B[i])

            # only one per row
            probM.addConstr(gp.quicksum(X[i,j] for j in range(J)) == 1, name = "row" + str(i))

            # cannot go along diagonals
            probM.addConstr(X[i,i] == 0)

        for j in range(J):
            # only one per column
            probM.addConstr(gp.quicksum(X[i,j] for i in range(I)) == 1, name = "col" + str(j))
        # Sub tour Elimination
        for i in range(I):
            for j in range(J):
                if i == j:
                    continue
                else:
                    probM.addConstr(TI[i] - TJ[j] + (B[i] - A[j] + D[i][j])*X[i,j] <= B[i] - A[j], name = "row" + str(i) + " - col" + str(j))


        probM.setParam(parameter, parakey)
        probM.optimize()

        params = {
        'Usecase' : 'Medium',
        'Parameter': parameter,
        'Parameter Value': parakey,
        'Obj Value': probM.ObjVal,
        'Runtime' : probM.Runtime
        }
        # Convert the dictionary to a dataframe
        params_df = pd.DataFrame(params, index=[0])
 
        # Append the dataframe to the result_df
        result_df = result_df.append(params_df, ignore_index=True)

        

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10706 rows, 10403 columns and 51206 nonzeros
Model fingerprint: 0xd2182b20
Variable types: 0 continuous, 10403 integer (10201 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 10519 rows and 5876 columns
Presolve time: 0.03s
Presolved: 187 rows, 4527 columns, 9061 nonzeros
Variable types: 0 continuous, 4527 integer (4527 binary)

Root relaxation: objective 5.997767e+02, 186 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     599.7766650  599.77666  0.00%     -    0s

Explored 1 nodes (186 simplex iterations) in 0.06 seconds (0.05 

C:\Users\workV\AppData\Local\Temp\ipykernel_24204\3154943429.py:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(params_df, ignore_index=True)


Set parameter BQPCuts to value 0
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10706 rows, 10403 columns and 51206 nonzeros
Model fingerprint: 0xd2182b20
Variable types: 0 continuous, 10403 integer (10201 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 10519 rows and 5876 columns
Presolve time: 0.03s
Presolved: 187 rows, 4527 columns, 9061 nonzeros
Variable types: 0 continuous, 4527 integer (4527 binary)

Root relaxation: objective 5.997767e+02, 186 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     599.7766650  599.77666  0.00%     -    0s

Explored 1 nodes (186 simplex i

C:\Users\workV\AppData\Local\Temp\ipykernel_24204\3154943429.py:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(params_df, ignore_index=True)


Set parameter BQPCuts to value 1
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10706 rows, 10403 columns and 51206 nonzeros
Model fingerprint: 0xd2182b20
Variable types: 0 continuous, 10403 integer (10201 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 10519 rows and 5876 columns
Presolve time: 0.03s
Presolved: 187 rows, 4527 columns, 9061 nonzeros
Variable types: 0 continuous, 4527 integer (4527 binary)

Root relaxation: objective 5.997767e+02, 186 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     599.7766650  599.77666  0.00%     -    0s

Explored 1 nodes (186 simplex i

...

C:\Users\workV\AppData\Local\Temp\ipykernel_24204\3154943429.py:238: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(params_df, ignore_index=True)


Set parameter GomoryPasses to value 98
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10706 rows, 10403 columns and 51206 nonzeros
Model fingerprint: 0xd2182b20
Variable types: 0 continuous, 10403 integer (10201 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 10519 rows and 5876 columns
Presolve time: 0.04s
Presolved: 187 rows, 4527 columns, 9061 nonzeros
Variable types: 0 continuous, 4527 integer (4527 binary)

Root relaxation: objective 5.997767e+02, 186 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     599.7766650  599.77666  0.00%     -    0s

Explored 1 nodes (186 sim

C:\Users\workV\AppData\Local\Temp\ipykernel_24204\3154943429.py:238: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(params_df, ignore_index=True)


Set parameter GomoryPasses to value 99
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10706 rows, 10403 columns and 51206 nonzeros
Model fingerprint: 0xd2182b20
Variable types: 0 continuous, 10403 integer (10201 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 10519 rows and 5876 columns
Presolve time: 0.04s
Presolved: 187 rows, 4527 columns, 9061 nonzeros
Variable types: 0 continuous, 4527 integer (4527 binary)

Root relaxation: objective 5.997767e+02, 186 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     599.7766650  599.77666  0.00%     -    0s

Explored 1 nodes (186 sim

C:\Users\workV\AppData\Local\Temp\ipykernel_24204\3154943429.py:238: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(params_df, ignore_index=True)


In [ ]:
BQPCuts
CliqueCuts
CoverCuts
FlowCoverCuts
FlowPathCuts
GUBCoverCuts
ImpliedCuts
InfProofCuts
LiftProjectCuts
MIPSepCuts
MIRCuts
ModKCuts
NetworkCuts
ProjImpliedCuts
PSDCuts
RelaxLiftCuts
RLTCuts
StrongCGCuts
SubMIPCuts
ZeroHalfCuts

In [4]:
#Large Case
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import *

parameters = [
    "BQPCuts",
    "CliqueCuts",
    "CoverCuts",
    "FlowCoverCuts",
    "FlowPathCuts",
    "GUBCoverCuts",
    "ImpliedCuts",
    "InfProofCuts",
    "LiftProjectCuts",
    "MIPSepCuts",
    "MIRCuts",
    "ModKCuts",
    "NetworkCuts",
    "ProjImpliedCuts",
    "PSDCuts",
    "RelaxLiftCuts",
    "RLTCuts",
    "StrongCGCuts",
    "SubMIPCuts",
    "ZeroHalfCuts"
]
for parameter in parameters:
    for parakey in [-1,0,1,2]:



        df = pd.read_csv(r"D:\Vibhu_Git_Projects\Optimization\Concert_TSPTW_Optimization\data\TSPTW_150.txt")


        all_vals = []
        for i in range(df.shape[0]):
            vals = [val for val in df.iloc[i,0].strip().split(" ") if val != ""]
            all_vals.append(vals)

        df_data = pd.DataFrame(all_vals, columns = ["CUST NO.", "X", "Y", "DEMAND", "READY TIME", "DUE DATE", "SERVICE TIME"])
        df_data = df_data.iloc[:-1,:]
        df_data["COORDINATES"] = df_data.apply(lambda row: (row["X"], row["Y"]), axis=1)
        df_data["DEMAND"] = np.random.randint(1,5,df_data.shape[0])
        df_data["SERVICE TIME"] = np.random.randint(1,9,df_data.shape[0])
        df_data = df_data.drop(columns=["X", "Y"])
        df_large = df_data.iloc[1:].sample(n = 120, replace = False, random_state=42).copy()
        initial = pd.DataFrame([df_data.iloc[0].to_list()], columns = df_data.columns)
        df_large = pd.concat([initial, df_large]).reset_index(drop=True)
        ### Create Distance and Time Matrices
        import math
        # Compute pairwise times matrix

        def distance(df, col, row1, row2):
            c1 = df.loc[row1, col]
            c2 = df.loc[row2, col]
            diff = (float(c1[0])-float(c2[0]), float(c1[1])-float(c2[1]))
            return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])


        large_dist = []
        for row1 in range(df_large.shape[0]):
            row_vals = []
            for row2 in range(df_large.shape[0]):
                row_vals.append(distance(df_large, "COORDINATES", row1, row2))
            large_dist.append(row_vals)
        dist_large = np.asarray(large_dist)
        I = len(dist_large)
        J = len(dist_large[0])

        # Start Times
        A = df_large["READY TIME"].astype(float).to_numpy()

        # End Times
        B = df_large["DUE DATE"].astype(float).to_numpy()

        # distance matrix
        D = dist_large
        probL = gp.Model("TSPTW - Concerts - Medium")

        X = probL.addVars(I, J, vtype=GRB.BINARY, name=[f"({i}), ({j})" for i in range(I) for j in range(J)])
        TI = probL.addVars(I, vtype=GRB.INTEGER, name=[f"TI_{i}" for i in range(I)])
        TJ = probL.addVars(J, vtype=GRB.INTEGER, name=[f"TJ_{j}" for j in range(J)])

        probL.setObjective(gp.quicksum(X[i,j]*D[i][j] for i in range(I) for j in range(J)), GRB.MINIMIZE)
        for i in range(I):
            # Initiation of Arrival to first node
            probL.addConstr(TI[i] - D[0][i]*X[0,i] >= 0)

            # start of time window
            probL.addConstr(TI[i] >= A[i])

            # end of time window
            probL.addConstr(TJ[i] <= B[i])

            # only one per row
            probL.addConstr(gp.quicksum(X[i,j] for j in range(J)) == 1, name = "row" + str(i))

            # cannot go along diagonals
            probL.addConstr(X[i,i] == 0)

        for j in range(J):
            # only one per column
            probL.addConstr(gp.quicksum(X[i,j] for i in range(I)) == 1, name = "col" + str(j))
        # Sub tour Elimination
        for i in range(I):
            for j in range(J):
                if i == j:
                    continue
                else:
                    probL.addConstr(TI[i] - TJ[j] + (B[i] - A[j] + D[i][j])*X[i,j] <= B[i] - A[j], name = "row" + str(i) + " - col" + str(j))

        probL.setParam(parameter, parakey)
        probL.optimize()

        params = {
        'Usecase' : 'Large',
        'Parameter': parameter,
        'Parameter Value': parakey,
        'Obj Value': probL.ObjVal,
        'Runtime' : probL.Runtime
        }
        # Convert the dictionary to a dataframe
        params_df = pd.DataFrame(params, index=[0])
 
        # Append the dataframe to the result_df
        result_df = result_df.append(params_df, ignore_index=True)

        
parameters = [
    "CutPasses",
    "CutAggPasses",
    "GomoryPasses"
]
for parameter in parameters:
    for parakey in range(100):
        

        df = pd.read_csv("./data/TSPTW_150.txt")


        all_vals = []
        for i in range(df.shape[0]):
            vals = [val for val in df.iloc[i,0].strip().split(" ") if val != ""]
            all_vals.append(vals)

        df_data = pd.DataFrame(all_vals, columns = ["CUST NO.", "X", "Y", "DEMAND", "READY TIME", "DUE DATE", "SERVICE TIME"])
        df_data = df_data.iloc[:-1,:]
        df_data["COORDINATES"] = df_data.apply(lambda row: (row["X"], row["Y"]), axis=1)
        df_data["DEMAND"] = np.random.randint(1,5,df_data.shape[0])
        df_data["SERVICE TIME"] = np.random.randint(1,9,df_data.shape[0])
        df_data = df_data.drop(columns=["X", "Y"])
        df_large = df_data.iloc[1:].sample(n = 120, replace = False, random_state=42).copy()
        initial = pd.DataFrame([df_data.iloc[0].to_list()], columns = df_data.columns)
        df_large = pd.concat([initial, df_large]).reset_index(drop=True)
        ### Create Distance and Time Matrices
        import math
        # Compute pairwise times matrix

        def distance(df, col, row1, row2):
            c1 = df.loc[row1, col]
            c2 = df.loc[row2, col]
            diff = (float(c1[0])-float(c2[0]), float(c1[1])-float(c2[1]))
            return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])


        large_dist = []
        for row1 in range(df_large.shape[0]):
            row_vals = []
            for row2 in range(df_large.shape[0]):
                row_vals.append(distance(df_large, "COORDINATES", row1, row2))
            large_dist.append(row_vals)
        dist_large = np.asarray(large_dist)
        I = len(dist_large)
        J = len(dist_large[0])

        # Start Times
        A = df_large["READY TIME"].astype(float).to_numpy()

        # End Times
        B = df_large["DUE DATE"].astype(float).to_numpy()

        # distance matrix
        D = dist_large
        probL = gp.Model("TSPTW - Concerts - Medium")

        X = probL.addVars(I, J, vtype=GRB.BINARY, name=[f"({i}), ({j})" for i in range(I) for j in range(J)])
        TI = probL.addVars(I, vtype=GRB.INTEGER, name=[f"TI_{i}" for i in range(I)])
        TJ = probL.addVars(J, vtype=GRB.INTEGER, name=[f"TJ_{j}" for j in range(J)])

        probL.setObjective(gp.quicksum(X[i,j]*D[i][j] for i in range(I) for j in range(J)), GRB.MINIMIZE)
        for i in range(I):
            # Initiation of Arrival to first node
            probL.addConstr(TI[i] - D[0][i]*X[0,i] >= 0)

            # start of time window
            probL.addConstr(TI[i] >= A[i])

            # end of time window
            probL.addConstr(TJ[i] <= B[i])

            # only one per row
            probL.addConstr(gp.quicksum(X[i,j] for j in range(J)) == 1, name = "row" + str(i))

            # cannot go along diagonals
            probL.addConstr(X[i,i] == 0)

        for j in range(J):
            # only one per column
            probL.addConstr(gp.quicksum(X[i,j] for i in range(I)) == 1, name = "col" + str(j))
        # Sub tour Elimination
        for i in range(I):
            for j in range(J):
                if i == j:
                    continue
                else:
                    probL.addConstr(TI[i] - TJ[j] + (B[i] - A[j] + D[i][j])*X[i,j] <= B[i] - A[j], name = "row" + str(i) + " - col" + str(j))

        probL.setParam(parameter, parakey)
        probL.optimize()

        params = {
        'Usecase' : 'Large',
        'Parameter': parameter,
        'Parameter Value': parakey,
        'Obj Value': probL.ObjVal,
        'Runtime' : probL.Runtime
        }
        # Convert the dictionary to a dataframe
        params_df = pd.DataFrame(params, index=[0])
 
        # Append the dataframe to the result_df
        result_df = result_df.append(params_df, ignore_index=True)


Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 15246 rows, 14883 columns and 73444 nonzeros
Model fingerprint: 0x7784225b
Variable types: 0 continuous, 14883 integer (14641 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 15026 rows and 8548 columns
Presolve time: 0.11s
Presolved: 220 rows, 6335 columns, 12670 nonzeros
Variable types: 0 continuous, 6335 integer (6335 binary)

Root relaxation: objective 6.440674e+02, 235 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     644.0673580  644.06736  0.00%     -    0s

Explored 1 nodes (235 simplex iterations) in 0.16 seconds (0.07

C:\Users\workV\AppData\Local\Temp\ipykernel_24204\745924150.py:128: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(params_df, ignore_index=True)


Set parameter BQPCuts to value 0
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 15246 rows, 14883 columns and 73444 nonzeros
Model fingerprint: 0x7784225b
Variable types: 0 continuous, 14883 integer (14641 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 15026 rows and 8548 columns
Presolve time: 0.04s
Presolved: 220 rows, 6335 columns, 12670 nonzeros
Variable types: 0 continuous, 6335 integer (6335 binary)

Root relaxation: objective 6.440674e+02, 235 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     644.0673580  644.06736  0.00%     -    0s

Explored 1 nodes (235 simplex 

C:\Users\workV\AppData\Local\Temp\ipykernel_24204\745924150.py:128: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(params_df, ignore_index=True)


Set parameter BQPCuts to value 1
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 15246 rows, 14883 columns and 73444 nonzeros
Model fingerprint: 0x7784225b
Variable types: 0 continuous, 14883 integer (14641 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 15026 rows and 8548 columns
Presolve time: 0.04s
Presolved: 220 rows, 6335 columns, 12670 nonzeros
Variable types: 0 continuous, 6335 integer (6335 binary)

Root relaxation: objective 6.440674e+02, 235 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     644.0673580  644.06736  0.00%     -    0s

Explored 1 nodes (235 simplex 

...

C:\Users\workV\AppData\Local\Temp\ipykernel_24204\745924150.py:234: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(params_df, ignore_index=True)


Set parameter GomoryPasses to value 98
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 15246 rows, 14883 columns and 73444 nonzeros
Model fingerprint: 0x7784225b
Variable types: 0 continuous, 14883 integer (14641 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 15026 rows and 8548 columns
Presolve time: 0.04s
Presolved: 220 rows, 6335 columns, 12670 nonzeros
Variable types: 0 continuous, 6335 integer (6335 binary)

Root relaxation: objective 6.440674e+02, 235 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     644.0673580  644.06736  0.00%     -    0s

Explored 1 nodes (235 si

C:\Users\workV\AppData\Local\Temp\ipykernel_24204\745924150.py:234: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(params_df, ignore_index=True)


Set parameter GomoryPasses to value 99
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 15246 rows, 14883 columns and 73444 nonzeros
Model fingerprint: 0x7784225b
Variable types: 0 continuous, 14883 integer (14641 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 15026 rows and 8548 columns
Presolve time: 0.04s
Presolved: 220 rows, 6335 columns, 12670 nonzeros
Variable types: 0 continuous, 6335 integer (6335 binary)

Root relaxation: objective 6.440674e+02, 235 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     644.0673580  644.06736  0.00%     -    0s

Explored 1 nodes (235 si

C:\Users\workV\AppData\Local\Temp\ipykernel_24204\745924150.py:234: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(params_df, ignore_index=True)


In [5]:
result_df

,Usecase,Parameter,Parameter Value,Obj Value,Runtime
0,Small,BQPCuts,-1,545.311159,0.073111
1,Small,BQPCuts,0,545.311159,0.055534
2,Small,BQPCuts,1,545.311159,0.059889
3,Small,BQPCuts,2,545.311159,0.050573
4,Small,CliqueCuts,-1,545.311159,0.047857
...,...,...,...,...,...
1135,Large,GomoryPasses,95,644.067358,0.088219
1136,Large,GomoryPasses,96,644.067358,0.078215
1137,Large,GomoryPasses,97,644.067358,0.125542
1138,Large,GomoryPasses,98,644.067358,0.074524


In [7]:
result_df.to_csv(r"D:\Vibhu_Git_Projects\Optimization\Concert_TSPTW_Optimization\data\OutputResult.csv")

In [9]:
# Small Case with model.tune
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import *

df = pd.read_csv(r"D:\Vibhu_Git_Projects\Optimization\Concert_TSPTW_Optimization\data\TSPTW_150.txt")
all_vals = []
for i in range(df.shape[0]):

    vals = [val for val in df.iloc[i,0].strip().split(" ") if val != ""]
    all_vals.append(vals)

df_data = pd.DataFrame(all_vals, columns = ["CUST NO.", "X", "Y", "DEMAND", "READY TIME", "DUE DATE", "SERVICE TIME"])
df_data = df_data.iloc[:-1,:]
df_data["COORDINATES"] = df_data.apply(lambda row: (row["X"], row["Y"]), axis=1)
df_data["DEMAND"] = np.random.randint(1,5,df_data.shape[0])
df_data["SERVICE TIME"] = np.random.randint(1,9,df_data.shape[0])
df_data = df_data.drop(columns=["X", "Y"])
df_small = df_data.iloc[1:].sample(n = 80, replace = False, random_state=42).copy()
initial = pd.DataFrame([df_data.iloc[0].to_list()], columns = df_data.columns)
df_small = pd.concat([initial, df_small]).reset_index(drop=True)
### Create Distance and Time Matrices
import math
# Compute pairwise times matrix

def distance(df, col, row1, row2):
    c1 = df.loc[row1, col]
    c2 = df.loc[row2, col]
    diff = (float(c1[0])-float(c2[0]), float(c1[1])-float(c2[1]))
    return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])

small_dist = []

for row1 in range(df_small.shape[0]):
    
    row_vals = []
    for row2 in range(df_small.shape[0]):

        row_vals.append(distance(df_small, "COORDINATES", row1, row2))
    small_dist.append(row_vals)

dist_small = np.asarray(small_dist)
### Model Implementation - Small
I = len(dist_small)
J = len(dist_small[0])

# Start Times
A = df_small["READY TIME"].astype(float).to_numpy()

# End Times
B = df_small["DUE DATE"].astype(float).to_numpy()

# distance matrix
D = dist_small
probS = gp.Model("TSPTW - Concerts - Small")

X = probS.addVars(I, J, vtype=GRB.BINARY, name=[f"({i}), ({j})" for i in range(I) for j in range(J)])
TI = probS.addVars(I, vtype=GRB.INTEGER, name=[f"TI_{i}" for i in range(I)])
TJ = probS.addVars(J, vtype=GRB.INTEGER, name=[f"TJ_{j}" for j in range(J)])

probS.setObjective(gp.quicksum(X[i,j]*D[i][j] for i in range(I) for j in range(J)), GRB.MINIMIZE)
for i in range(I):
    # Initiation of Arrival to first node
    probS.addConstr(TI[i] - D[0][i]*X[0,i] >= 0)

    # start of time window
    probS.addConstr(TI[i] >= A[i])

    # end of time window
    probS.addConstr(TJ[i] <= B[i])

    # only one per row
    probS.addConstr(gp.quicksum(X[i,j] for j in range(J)) == 1, name = "row" + str(i))

    # cannot go along diagonals
    probS.addConstr(X[i,i] == 0)

for j in range(J):
    # only one per column
    probS.addConstr(gp.quicksum(X[i,j] for i in range(I)) == 1, name = "col" + str(j))
# Sub tour Elimination
for i in range(I):
    for j in range(J):
        if i == j:
            continue
        else:
            probS.addConstr(TI[i] - TJ[j] + (B[i] - A[j] + D[i][j])*X[i,j] <= B[i] - A[j], name = "row" + str(i) + " - col" + str(j))
probS.tune()





Solving model using baseline parameter set with TimeLimit=3600s

Testing candidate parameter set 1...

	Default parameters

Solving with random seed #1 ...
Optimize a model with 6966 rows, 6723 columns and 32966 nonzeros
Model fingerprint: 0x1b1213af
Variable types: 0 continuous, 6723 integer (6561 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 6815 rows and 3764 columns
Presolve time: 0.02s
Presolved: 151 rows, 2959 columns, 5925 nonzeros
Variable types: 0 continuous, 2959 integer (2959 binary)

Root relaxation: objective 5.453112e+02, 161 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     545.3111592  545.31116  0.00%     -    0s

Explored 1 nodes (161 simplex iterations) i

In [11]:
#Medium Case with model.tune()
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import *
### Read in Source Data and Create 3 Scenarios
df = pd.read_csv(r"D:\Vibhu_Git_Projects\Optimization\Concert_TSPTW_Optimization\data\TSPTW_150.txt")


all_vals = []
for i in range(df.shape[0]):
    vals = [val for val in df.iloc[i,0].strip().split(" ") if val != ""]
    all_vals.append(vals)

df_data = pd.DataFrame(all_vals, columns = ["CUST NO.", "X", "Y", "DEMAND", "READY TIME", "DUE DATE", "SERVICE TIME"])
df_data = df_data.iloc[:-1,:]
df_data["COORDINATES"] = df_data.apply(lambda row: (row["X"], row["Y"]), axis=1)
df_data["DEMAND"] = np.random.randint(1,5,df_data.shape[0])
df_data["SERVICE TIME"] = np.random.randint(1,9,df_data.shape[0])
df_data = df_data.drop(columns=["X", "Y"])
df_medium = df_data.iloc[1:].sample(n = 100, replace = False, random_state=42).copy()
initial = pd.DataFrame([df_data.iloc[0].to_list()], columns = df_data.columns)
df_medium = pd.concat([initial, df_medium]).reset_index(drop=True)
### Create Distance and Time Matrices
import math
# Compute pairwise times matrix

def distance(df, col, row1, row2):
    c1 = df.loc[row1, col]
    c2 = df.loc[row2, col]
    diff = (float(c1[0])-float(c2[0]), float(c1[1])-float(c2[1]))
    return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])


medium_dist = []
for row1 in range(df_medium.shape[0]):
    row_vals = []
    for row2 in range(df_medium.shape[0]):
        row_vals.append(distance(df_medium, "COORDINATES", row1, row2))
    medium_dist.append(row_vals)
dist_medium = np.asarray(medium_dist)


I = len(dist_medium)
J = len(dist_medium[0])

# Start Times
A = df_medium["READY TIME"].astype(float).to_numpy()

# End Times
B = df_medium["DUE DATE"].astype(float).to_numpy()

# distance matrix
D = dist_medium
probM = gp.Model("TSPTW - Concerts - Medium")

X = probM.addVars(I, J, vtype=GRB.BINARY, name=[f"({i}), ({j})" for i in range(I) for j in range(J)])
TI = probM.addVars(I, vtype=GRB.INTEGER, name=[f"TI_{i}" for i in range(I)])
TJ = probM.addVars(J, vtype=GRB.INTEGER, name=[f"TJ_{j}" for j in range(J)])

probM.setObjective(gp.quicksum(X[i,j]*D[i][j] for i in range(I) for j in range(J)), GRB.MINIMIZE)
for i in range(I):
    # Initiation of Arrival to first node
    probM.addConstr(TI[i] - D[0][i]*X[0,i] >= 0)

    # start of time window
    probM.addConstr(TI[i] >= A[i])

    # end of time window
    probM.addConstr(TJ[i] <= B[i])

    # only one per row
    probM.addConstr(gp.quicksum(X[i,j] for j in range(J)) == 1, name = "row" + str(i))

    # cannot go along diagonals
    probM.addConstr(X[i,i] == 0)

for j in range(J):
    # only one per column
    probM.addConstr(gp.quicksum(X[i,j] for i in range(I)) == 1, name = "col" + str(j))
# Sub tour Elimination
for i in range(I):
    for j in range(J):
        if i == j:
            continue
        else:
            probM.addConstr(TI[i] - TJ[j] + (B[i] - A[j] + D[i][j])*X[i,j] <= B[i] - A[j], name = "row" + str(i) + " - col" + str(j))


probM.tune()





Solving model using baseline parameter set with TimeLimit=3600s

Testing candidate parameter set 1...

	Default parameters

Solving with random seed #1 ...
Optimize a model with 10706 rows, 10403 columns and 51206 nonzeros
Model fingerprint: 0xd2182b20
Variable types: 0 continuous, 10403 integer (10201 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 10519 rows and 5876 columns
Presolve time: 0.05s
Presolved: 187 rows, 4527 columns, 9061 nonzeros
Variable types: 0 continuous, 4527 integer (4527 binary)

Root relaxation: objective 5.997767e+02, 186 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     599.7766650  599.77666  0.00%     -    0s

Explored 1 nodes (186 simplex iteratio

In [14]:
#Large Case with model.tune()
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import *
df = pd.read_csv(r"D:\Vibhu_Git_Projects\Optimization\Concert_TSPTW_Optimization\data\TSPTW_150.txt")


all_vals = []
for i in range(df.shape[0]):
    vals = [val for val in df.iloc[i,0].strip().split(" ") if val != ""]
    all_vals.append(vals)

df_data = pd.DataFrame(all_vals, columns = ["CUST NO.", "X", "Y", "DEMAND", "READY TIME", "DUE DATE", "SERVICE TIME"])
df_data = df_data.iloc[:-1,:]
df_data["COORDINATES"] = df_data.apply(lambda row: (row["X"], row["Y"]), axis=1)
df_data["DEMAND"] = np.random.randint(1,5,df_data.shape[0])
df_data["SERVICE TIME"] = np.random.randint(1,9,df_data.shape[0])
df_data = df_data.drop(columns=["X", "Y"])
df_large = df_data.iloc[1:].sample(n = 120, replace = False, random_state=42).copy()
initial = pd.DataFrame([df_data.iloc[0].to_list()], columns = df_data.columns)
df_large = pd.concat([initial, df_large]).reset_index(drop=True)
### Create Distance and Time Matrices
import math
# Compute pairwise times matrix

def distance(df, col, row1, row2):
    c1 = df.loc[row1, col]
    c2 = df.loc[row2, col]
    diff = (float(c1[0])-float(c2[0]), float(c1[1])-float(c2[1]))
    return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])


large_dist = []
for row1 in range(df_large.shape[0]):
    row_vals = []
    for row2 in range(df_large.shape[0]):
        row_vals.append(distance(df_large, "COORDINATES", row1, row2))
    large_dist.append(row_vals)
dist_large = np.asarray(large_dist)
I = len(dist_large)
J = len(dist_large[0])

# Start Times
A = df_large["READY TIME"].astype(float).to_numpy()

# End Times
B = df_large["DUE DATE"].astype(float).to_numpy()

# distance matrix
D = dist_large
probL = gp.Model("TSPTW - Concerts - Medium")

X = probL.addVars(I, J, vtype=GRB.BINARY, name=[f"({i}), ({j})" for i in range(I) for j in range(J)])
TI = probL.addVars(I, vtype=GRB.INTEGER, name=[f"TI_{i}" for i in range(I)])
TJ = probL.addVars(J, vtype=GRB.INTEGER, name=[f"TJ_{j}" for j in range(J)])

probL.setObjective(gp.quicksum(X[i,j]*D[i][j] for i in range(I) for j in range(J)), GRB.MINIMIZE)
for i in range(I):
    # Initiation of Arrival to first node
    probL.addConstr(TI[i] - D[0][i]*X[0,i] >= 0)

    # start of time window
    probL.addConstr(TI[i] >= A[i])

    # end of time window
    probL.addConstr(TJ[i] <= B[i])

    # only one per row
    probL.addConstr(gp.quicksum(X[i,j] for j in range(J)) == 1, name = "row" + str(i))

    # cannot go along diagonals
    probL.addConstr(X[i,i] == 0)

for j in range(J):
    # only one per column
    probL.addConstr(gp.quicksum(X[i,j] for i in range(I)) == 1, name = "col" + str(j))
# Sub tour Elimination
for i in range(I):
    for j in range(J):
        if i == j:
            continue
        else:
            probL.addConstr(TI[i] - TJ[j] + (B[i] - A[j] + D[i][j])*X[i,j] <= B[i] - A[j], name = "row" + str(i) + " - col" + str(j))

probL.tune()




Solving model using baseline parameter set with TimeLimit=3600s

Testing candidate parameter set 1...

	Default parameters

Solving with random seed #1 ...
Optimize a model with 15246 rows, 14883 columns and 73444 nonzeros
Model fingerprint: 0x7784225b
Variable types: 0 continuous, 14883 integer (14641 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+03]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 15026 rows and 8548 columns
Presolve time: 0.06s
Presolved: 220 rows, 6335 columns, 12670 nonzeros
Variable types: 0 continuous, 6335 integer (6335 binary)

Root relaxation: objective 6.440674e+02, 235 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     644.0673580  644.06736  0.00%     -    0s

Explored 1 nodes (235 simplex iterati